# Lab 5. Accessing OCI Gen AI Service with Python SDK.

Oracle Cloud Infrastructure (OCI) Generative AI is a fully managed service that provides a set of state-of-the-art, customizable large language models (LLMs) that cover a wide range of use cases, and which is available through a single API.
Using the OCI Generative AI service you can access ready-to-use pretrained models, or create and host your own fine-tuned custom models based on your own data on dedicated AI clusters. Detailed documentation of the service and API is available __[here](https://docs.oracle.com/en-us/iaas/Content/generative-ai/home.htm)__ and __[here](https://docs.oracle.com/en-us/iaas/api/#/en/generative-ai/20231130/)__.

In this lab, we shall explore how to use OCI's Genrative AI models.

## Setup
- The first step is to import OS and oci libraries and set the Endpoint and Model IDs.
- Ensure that you have **oci cli** configured prior to this lab.

# Important : Before Executing this Lab Configure OCI CLI. 

Steps :
- Tenancy OCID : ocid1.tenancy.oc1..aaaaaaaafzvkfj24vlsyzt7gvue6a6owepu5ghd5bvywtfopf2fnratinxvq
- Region : us-ashburn-1
1.	Launch the terminal  in your Data Science Notebook.
2.	Run the Configuration with command:
`# oci setup config`
3.	Press Enter when prompted for directory name to accept the default. 
Enter a location for your config [~/.oci/config]:
You will be prompted to enter user OCID and tenancy OCID (ocid1.tenancy.oc1..aaaaaaaafzvkfj24vlsyzt7gvue6a6owepu5ghd5bvywtfopf2fnratinxvq).
4.	Login to Oracle Cloud infrastructure at https://cloud.oracle.com. Click user icon and click User Settings. 
5.	In User settings click Copy next to OCID for your user name
6.	Switch to the terminal session and paste the user OCID using mouse/touch pad and press Enter. You will be prompted to Enter Tenancy OCID
7.	Switch to OCI Console window, Click user icon and click your Tenancy name, copy the OCID. Also note down your region (for me “us-ashburn-1”)
8.	Switch to the SSH terminal window and paste the tenancy OCID using mouse/touch pad and press Enter. You will be prompted to enter your region.
9.	Type your region and press Enter. Enter Y for ‘New RSA key pair’. Press Enter and accept default options for directories. Press Enter when prompted for passphrase.
10.	Try these commands and Verify the API key files and OCI CLI config files exist
`# cat ~/.oci/config`

11. Copy the contents of ~/.oci/oci_api_key_public.pem  to **User (Top Right Corner of OCI Web Console) -> My Profile - > API Keys - > Paste Public Key**


You will also need to install the OCI sdk

In [ ]:
!pip install oci oci-cli -qU

In [ ]:
import oci

# Please update config profile name and use the compartment that has policies grant permissions for using Generative AI Service
compartment_id = "ocid1.compartment.oc1..aaaaaaaah2ho2v6m7slojkzc5fcb7fo33kvct7gn2pqlwugjxwmiov5mnf4a"
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('~/.oci/config', CONFIG_PROFILE)

# Service endpoint
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

# Model OCID
model_id = "ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyawk6mgunzodenakhkuwxanvt6wo3jcpf72ln52dymk4wq"

# Generative AI Service Client
generative_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))

#### Helper function
We will use OCI GenAI's `cohere.command-r-16k` model. 

Create a helper function that will make it easier to use prompts and look at the generated outputs.  

In [ ]:
def get_completion(prompt):
  cd = oci.generative_ai_inference.models.ChatDetails()

  cr = oci.generative_ai_inference.models.CohereChatRequest()
  cr.message = prompt
  cr.max_tokens = 600
  cr.temperature = 0
  cr.frequency_penalty = 1
  cr.top_p = 0.75
  cr.top_k = 0

  cd.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id=model_id)
  cd.chat_request = cr
  cd.compartment_id = compartment_id
  cr = generative_client.chat(cd)

  return (cr.data.chat_response.text)

# Create a Prompt and Make an API Call
### Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [ ]:
text = f"""
You should express what you want a model to do by providing instructions that are as clear and \
specific as you can possibly make them. This will guide the model towards the desired output, \
and reduce the chances of receiving irrelevant or incorrect responses. Don't confuse writing a \
clear prompt with writing a short prompt. In many cases, longer prompts provide more clarity \
and context for the model, which can lead to more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks into a single sentence.
```{text}```
"""
response = get_completion(prompt)
print(response)

## Explore All Examples from Lab 2 and verify the output.

## Sequence 1. Summarizing Text
#### a. Text to summarize

In [ ]:
prod_review = """
Got this panda plush toy for my daughter's birthday, who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's a bit small for what I paid though. I think there \ 
might be other options that are bigger for the same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it to her.
"""

### b. Summarize with a word/sentence/character limit

In [ ]:
prompt = f"""
Your task is to generate a short summary of a product review from an ecommerce site. 

Summarize the review below, delimited by triple backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


### c. Summarize with a focus on shipping and delivery

In [ ]:
prompt = f"""
Your task is to generate a short summary of a product review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)


### d. Summarize with a focus on price and value

In [ ]:
prompt = f"""
Your task is to generate a short summary of a product review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the price of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects that are relevant to the price and perceived value. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

#### Summaries include topics that are not related to the topic of focus.
### e. Try "extract" instead of "summarize"

In [ ]:
prompt = f"""
Your task is to extract relevant information from a product review from an ecommerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

### f. Summarize multiple product reviews

In [ ]:
review_1 = prod_review 

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers and products. 
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist every day. My teeth feel sparkly clean! 
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about two days.
"""

reviews = [review_1, review_2, review_3, review_4]

In [ ]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product review from an ecommerce site. 

    Summarize the review below, delimited by triple backticks in at most 20 words. 

    Review: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")


## Sequence 2. Inferring with Cohere `cohere.command-r-16k` Model
Infer sentiment and topics from product reviews and news articles.

### a. Define the Review

In [ ]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

### b. Sentiment (positive/negative)

In [ ]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

### c. Identify types of emotions

In [ ]:
prompt = f"""
Identify a list of emotions that the writer of the following review is expressing. Include no more than \
five items in the list. Format your answer as a list of lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

### d. Identify anger

In [ ]:
prompt = f"""
Is the writer of the following review expressing anger? The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

### e. Extract product and company name from customer reviews

In [ ]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

### f. Doing multiple tasks at once

In [ ]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

### g. Inferring topics

In [ ]:
story = """
In a recent survey conducted by the government, public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the Social Security Administration had the lowest satisfaction rating, 
with only 45% of employees indicating they were satisfied with their job. The government has pledged to address
the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

### h. Infer 5 topics

In [ ]:
prompt = f"""
Determine five topics that are being discussed in the following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

In [ ]:
response.split(sep=',')

In [ ]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

### i. Make a news alert for certain topics

In [ ]:
prompt = f"""
Determine whether each item in the following list of topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)

## Sequence 3. Generate a marketing product description from a product fact sheet


**a. The product fact sheet**

In [ ]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [ ]:
prompt = f"""
Your task is to help a marketing team create a description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)